### This notebook is fourth-fifth submission for the hackathon 

https://datahack.analyticsvidhya.com/contest/amexpert-decipher-women-machine-learning-hackathon/

My public and private leaderboard scores are 132.4390534342 and 132.8241022151 respectively

In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns',100)
import warnings
warnings.filterwarnings('ignore')

In [32]:
from statistics import mode 
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import mean_squared_log_error,mean_squared_error
import lightgbm as lgbm
from sklearn.linear_model import LinearRegression,RANSACRegressor,Ridge
from sklearn.svm import SVR

In [3]:
train = pd.read_csv('/Users/lavanyamk/Documents/decipHER/train_bFQbE3f/train.csv')
train_dict = pd.read_excel('/Users/lavanyamk/Documents/decipHER/train_bFQbE3f/Data_Dictionary.xlsx')

In [4]:
test = pd.read_csv('/Users/lavanyamk/Documents/decipHER/test_9K3DBWQ.csv')

In [5]:
train.columns

Index(['id', 'account_type', 'gender', 'age', 'region_code', 'cc_cons_apr',
       'dc_cons_apr', 'cc_cons_may', 'dc_cons_may', 'cc_cons_jun',
       'dc_cons_jun', 'cc_count_apr', 'cc_count_may', 'cc_count_jun',
       'dc_count_apr', 'dc_count_may', 'dc_count_jun', 'card_lim',
       'personal_loan_active', 'vehicle_loan_active', 'personal_loan_closed',
       'vehicle_loan_closed', 'investment_1', 'investment_2', 'investment_3',
       'investment_4', 'debit_amount_apr', 'credit_amount_apr',
       'debit_count_apr', 'credit_count_apr', 'max_credit_amount_apr',
       'debit_amount_may', 'credit_amount_may', 'credit_count_may',
       'debit_count_may', 'max_credit_amount_may', 'debit_amount_jun',
       'credit_amount_jun', 'credit_count_jun', 'debit_count_jun',
       'max_credit_amount_jun', 'loan_enq', 'emi_active', 'cc_cons'],
      dtype='object')

In [6]:
train_dict

,Variable,Description
0,id,Unique ID for every Customer
1,account_type,Account Type – current or saving
2,gender,Gender of customer
3,age,Age of customer
4,region_code,Code assigned to region of residence (has order)
5,cc_cons_apr,Credit card spend in April
6,dc_cons_apr,Debit card spend in April
7,cc_cons_may,Credit card spend in May
8,dc_cons_may,Debit card spend in May
9,cc_cons_jun,Credit card spend in June


In [7]:
test.shape

(14067, 43)

In [8]:
train.shape

(32820, 44)

In [9]:
train.card_lim.isna().value_counts()

False    32811
True         9
Name: card_lim, dtype: int64

In [10]:
train.describe()

,id,age,region_code,cc_cons_apr,dc_cons_apr,cc_cons_may,dc_cons_may,cc_cons_jun,dc_cons_jun,cc_count_apr,cc_count_may,cc_count_jun,dc_count_apr,dc_count_may,dc_count_jun,card_lim,personal_loan_active,vehicle_loan_active,personal_loan_closed,vehicle_loan_closed,investment_1,investment_2,investment_3,investment_4,debit_amount_apr,credit_amount_apr,debit_count_apr,credit_count_apr,max_credit_amount_apr,debit_amount_may,credit_amount_may,credit_count_may,debit_count_may,max_credit_amount_may,debit_amount_jun,credit_amount_jun,credit_count_jun,debit_count_jun,max_credit_amount_jun,emi_active,cc_cons
count,32820.000000,32820.000000,32820.000000,32820.000000,13768.000000,3.282000e+04,15481.000000,32820.000000,11523.000000,30421.000000,31667.000000,31230.000000,13768.000000,15481.000000,17323.000000,32811.000000,2742.0,846.0,2883.0,1625.0,1.426000e+03,2.338000e+03,1.332000e+03,406.000000,3.093000e+04,2.951800e+04,31062.000000,31062.000000,2.951800e+04,3.051900e+04,2.941500e+04,30749.000000,30749.000000,2.941500e+04,3.062500e+04,3.127700e+04,31292.000000,31292.000000,3.127700e+04,3.282000e+04,32820.000000
mean,23450.929616,35.951676,554.854327,16661.907189,5727.615837,1.441190e+04,5635.506904,9031.179889,4666.713420,9.162191,9.540184,12.298687,5.256900,6.267618,7.897016,121553.749657,1.0,1.0,1.0,1.0,8.115118e+05,1.765459e+05,3.975786e+04,4972.642169,7.002974e+04,7.541190e+04,14.887290,3.791771,4.647113e+04,6.709618e+04,7.959347e+04,4.087352,13.934795,4.969688e+04,7.759673e+04,7.583531e+04,5.767736,16.001438,4.474355e+04,9.567620e+03,6824.631840
std,13495.676061,8.529949,188.220317,25138.435416,11544.058014,2.363688e+04,14052.343253,15720.847307,10265.189742,10.790714,9.648387,11.714253,6.147011,7.380032,9.197672,123019.826520,0.0,0.0,0.0,0.0,4.418812e+06,4.403071e+05,8.430780e+04,6107.212042,1.641944e+05,1.777199e+05,11.589652,3.482266,1.181413e+05,1.855331e+05,2.026879e+05,3.847766,11.455178,1.287256e+05,3.140103e+05,3.062791e+05,4.704716,13.370195,1.215379e+05,2.616736e+05,12459.140593
min,0.000000,22.000000,123.000000,233.970000,10.010000,1.030000e+02,20.000000,51.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.0,1.0,1.0,1.0,2.000000e-02,3.000000e+00,0.000000e+00,-12179.000000,4.010000e+00,3.432000e+01,0.000000,0.000000,1.800000e-01,1.500000e-01,1.061000e+01,0.000000,0.000000,1.500000e-01,5.230000e+00,1.188000e+01,0.000000,0.000000,6.000000e-02,0.000000e+00,0.000000
25%,11824.750000,31.000000,425.000000,3712.000000,849.000000,2.983298e+03,796.000000,1637.875000,743.000000,3.000000,3.000000,4.000000,1.000000,2.000000,2.000000,45000.000000,1.0,1.0,1.0,1.0,4.885100e+03,3.500000e+04,6.971250e+03,977.000000,2.229260e+04,2.531025e+04,6.000000,2.000000,1.800000e+04,2.010401e+04,2.467500e+04,2.000000,6.000000,1.817300e+04,2.509642e+04,2.306400e+04,3.000000,6.000000,1.600000e+04,0.000000e+00,1285.000000
50%,23478.500000,34.000000,575.000000,8912.790000,2282.160000,7.332298e+03,2129.000000,4147.487500,1832.000000,7.000000,7.000000,9.000000,3.000000,4.000000,5.000000,75000.000000,1.0,1.0,1.0,1.0,3.289308e+04,8.500000e+04,2.500000e+04,5427.500000,4.114024e+04,4.394596e+04,12.000000,3.000000,2.813100e+04,3.836849e+04,4.538700e+04,3.000000,11.000000,3.000000e+04,4.593499e+04,4.295500e+04,5.000000,13.000000,2.687600e+04,1.510650e+02,3141.000000
75%,35096.250000,39.000000,683.000000,19932.060000,6074.600000,1.684785e+04,5640.000000,10126.250000,4607.500000,12.000000,13.000000,17.000000,7.000000,8.000000,10.000000,145000.000000,1.0,1.0,1.0,1.0,1.748331e+05,1.763457e+05,5.000000e+04,6885.000000,7.541069e+04,7.975369e+04,20.000000,5.000000,4.800000e+04,7.373561e+04,8.917050e+04,5.000000,19.000000,5.000000e+04,8.413612e+04,8.072400e+04,7.000000,22.000000,4.701000e+04,4.176868e+03,7349.250000
max,46886.000000,224.000000,977.000000,804712.580000,366223.160000,1.117246e+06,911223.490000,382914.100000,301991.940000,1012.000000,601.000000,653.000000,82.000000,97.000000,104.000000,1000000.000000,1.0,1.0,1.0,1.0,8.445222e+07,

In [11]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14067 entries, 0 to 14066
Data columns (total 43 columns):
id                       14067 non-null int64
account_type             14067 non-null object
gender                   14067 non-null object
age                      14067 non-null int64
region_code              14067 non-null int64
cc_cons_apr              14067 non-null float64
dc_cons_apr              5837 non-null float64
cc_cons_may              14067 non-null float64
dc_cons_may              6692 non-null float64
cc_cons_jun              14067 non-null float64
dc_cons_jun              5003 non-null float64
cc_count_apr             13029 non-null float64
cc_count_may             13525 non-null float64
cc_count_jun             13433 non-null float64
dc_count_apr             5837 non-null float64
dc_count_may             6692 non-null float64
dc_count_jun             7515 non-null float64
card_lim                 14064 non-null float64
personal_loan_active     1134 non-null fl

In [12]:
train_init = train[['id','age','gender','cc_cons_apr','cc_cons_may','cc_cons_jun','card_lim']]
test_init = test[['id','age','gender','cc_cons_apr','cc_cons_may','cc_cons_jun','card_lim']]

In [13]:
#
train_init['con_avg_all'] = (train_init['cc_cons_jun']+train_init['cc_cons_may']+train_init['cc_cons_apr'])/3
test_init['con_avg_all'] = (test_init['cc_cons_jun']+test_init['cc_cons_may']+test_init['cc_cons_apr'])/3
#
train_init['con_avg_jun_may'] =(train_init['cc_cons_jun']+train_init['cc_cons_may'])/2
test_init['con_avg_jun_may'] =(test_init['cc_cons_jun']+test_init['cc_cons_may'])/2
#
train_init['con_avg_apr_may'] =(train_init['cc_cons_apr']+train_init['cc_cons_may'])/2
test_init['con_avg_apr_may'] =(test_init['cc_cons_apr']+test_init['cc_cons_may'])/2
#
train_init['con_avg_of_avg'] =(train_init['con_avg_all']+ train_init['con_avg_jun_may']+train_init['con_avg_apr_may'])/3
test_init['con_avg_of_avg'] =(test_init['con_avg_all']+test_init['con_avg_jun_may'] + test_init['con_avg_apr_may'])/3
#
train_init['cc_con_std_all'] = train_init[['cc_cons_apr','cc_cons_may','cc_cons_jun']].std(axis=1)
test_init['cc_con_std_all'] = test_init[['cc_cons_apr','cc_cons_may','cc_cons_jun']].std(axis=1)
#
train_init['transactions_jun'] = train['debit_amount_jun']+train['credit_amount_jun']
train_init['transactions_may'] = train['debit_amount_may']+train['credit_amount_may']
train_init['transactions_apr'] = train['debit_amount_apr']+train['credit_amount_apr']
test_init['transactions_jun'] = test['debit_amount_jun']+test['credit_amount_jun']
test_init['transactions_may'] = test['debit_amount_may']+test['credit_amount_may']
test_init['transactions_apr'] = test['debit_amount_apr']+test['credit_amount_apr']
#
train_init['transactions_count_jun'] = train['debit_count_jun']+train['credit_count_jun']
train_init['transactions_count_may'] = train['debit_count_may']+train['credit_count_may']
train_init['transactions_count_apr'] = train['debit_count_apr']+train['credit_count_apr']
test_init['transactions_count_jun'] = test['debit_count_jun']+test['credit_count_jun']
test_init['transactions_count_may'] = test['debit_count_may']+test['credit_count_may']
test_init['transactions_count_apr'] = test['debit_count_apr']+test['credit_count_apr']
#
train_init['transactions_freq'] = (train_init['transactions_jun'] + train_init['transactions_may'] + train_init['transactions_apr'])/(train_init['transactions_count_jun']+train_init['transactions_count_may']+train_init['transactions_count_apr'])
test_init['transactions_freq'] = (test_init['transactions_jun'] + test_init['transactions_may'] + test_init['transactions_apr'])/(test_init['transactions_count_jun']+test_init['transactions_count_may']+test_init['transactions_count_apr'])
train_init['transactions_debit_freq'] = (train['debit_amount_jun'] + train['debit_amount_may'] + train['debit_amount_apr'])/(train['debit_count_jun']+train['debit_count_may']+train['debit_count_apr'])
test_init['transactions_debit_freq'] = (test['debit_amount_jun'] + test['debit_amount_may'] + test['debit_amount_apr'])/(test['debit_count_jun']+test['debit_count_may']+test['debit_count_apr'])
train_init['transactions_credit_freq'] = (train['credit_amount_jun'] + train['credit_amount_may'] + train['credit_amount_apr'])/(train['credit_count_jun']+train['credit_count_may']+train['credit_count_apr'])
test_init['transactions_credit_freq'] = (test['credit_amount_jun'] + test['credit_amount_may'] + test['credit_amount_apr'])/(test['credit_count_jun']+test['credit_count_may']+test['credit_count_apr'])


In [14]:
#train_init['cc_con_std'] = train_init[['cc_cons_apr','cc_cons_may','cc_cons_jun']].std(axis=1)

In [15]:
pd.concat([train_init[['age','cc_cons_apr','cc_cons_may','cc_cons_jun','con_avg_all',
                       'con_avg_jun_may','con_avg_apr_may','con_avg_of_avg',
                       'cc_con_std_all','transactions_jun','transactions_may',
                       'transactions_apr','transactions_freq']],train['cc_cons']],axis=1).head(20)


,age,cc_cons_apr,cc_cons_may,cc_cons_jun,con_avg_all,con_avg_jun_may,con_avg_apr_may,con_avg_of_avg,cc_con_std_all,transactions_jun,transactions_may,transactions_apr,transactions_freq,cc_cons
0,63,7695.00,479.00,622.00,2932.000000,550.500,4087.000,2523.166667,4125.498637,21495.22,138992.22,101464.47,13786.942632,435
1,33,52702.00,29873.00,1087.00,27887.333333,15480.000,41287.500,28218.277778,25864.729079,177427.66,277708.85,328862.76,10888.878750,1921
2,44,10800.49,21659.64,1168.00,11209.376667,11413.820,16230.065,12951.087222,10251.937315,76124.91,38334.00,90426.55,3794.175185,1970
3,44,20936.00,2094.00,11778.50,11602.833333,6936.250,11515.000,10018.027778,9422.228244,12970.55,38890.24,4275.07,6237.317778,21579
4,35,1183.83,3656.29,945.00,1928.373333,2300.645,2420.060,2216.359444,1501.176854,79661.05,10655.00,16733.00,5352.452500,1814
5,26,53819.88,4712.92,1048.00,19860.266667,2880.460,29266.400,17335.708889,29466.920656,64372.39,52578.50,15509.72,1576.912024,626
6,30,1495.00,2212.00,1964.00,1890.333333,2088.000,1853.500,1943.944444,364.132302,109501.33,43889.00,44515.00,2301.224767,2052
7,32,1847.00,1347.80,669.00,1287.933333,1008.400,1597.400,1297.911111,591.277442,73652.36,61600.55,76799.23,2524.430238,434
8,37,9214.92,95625.88,11419.76,38753.520000,53522.820,52420.400,48232.246667,49265.244634,540602.50,251299.38,141013.05,6219.432867,10331
9,47,9191.31,8327.55,14642.00,10720.286667,11484.775,8759.430,10321.497222,3423.652572,NaN,NaN,NaN,NaN,2437


In [16]:
train_init.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32820 entries, 0 to 32819
Data columns (total 21 columns):
id                          32820 non-null int64
age                         32820 non-null int64
gender                      32820 non-null object
cc_cons_apr                 32820 non-null float64
cc_cons_may                 32820 non-null float64
cc_cons_jun                 32820 non-null float64
card_lim                    32811 non-null float64
con_avg_all                 32820 non-null float64
con_avg_jun_may             32820 non-null float64
con_avg_apr_may             32820 non-null float64
con_avg_of_avg              32820 non-null float64
cc_con_std_all              32820 non-null float64
transactions_jun            30610 non-null float64
transactions_may            29185 non-null float64
transactions_apr            29386 non-null float64
transactions_count_jun      31292 non-null float64
transactions_count_may      30749 non-null float64
transactions_count_apr      3

In [17]:
pd.concat([train['card_lim'],train_init['card_lim']],axis=1)
train_init.loc[train.card_lim.isna(),'card_lim']

3813    NaN
8406    NaN
13822   NaN
15833   NaN
18496   NaN
25159   NaN
29953   NaN
30640   NaN
31691   NaN
Name: card_lim, dtype: float64

In [ ]:
pd.concat([train.card_lim,train_init.card_lim],axis=1)
pd.concat([test.card_lim,test_init.card_lim],axis=1)

In [19]:
ages = [20, 30, 40, 50, 60, 71, 250]
for i in range(len(ages) - 1):
    temp_train = train_init[(train_init.age > ages[i]) & (train_init.age <= ages[i + 1])]
    train_init.loc[(train_init.age > ages[i]) & (train_init.age <= ages[i + 1]),'card_lim'] = temp_train['card_lim'].fillna(mode(temp_train['card_lim']))
    train_init.loc[(train_init.age > ages[i]) & (train_init.age <= ages[i + 1]) & (train.card_lim==0),'card_lim'] = mode(train_init.loc[(train_init.age > ages[i]) & (train_init.age <= ages[i + 1]),'card_lim'])
    temp_test = test_init[(test_init.age > ages[i]) & (test_init.age <= ages[i + 1])]
    test_init.loc[(test_init.age > ages[i]) & (test_init.age <= ages[i + 1]),'card_lim'] = temp_test['card_lim'].fillna(mode(temp_test['card_lim']))
    test_init.loc[(test_init.age > ages[i]) & (test_init.age <= ages[i + 1]) & (test.card_lim==0),'card_lim'] = mode(test_init.loc[(test_init.age > ages[i]) & (test_init.age <= ages[i + 1]),'card_lim'])

In [20]:
train_init[['investment_1', 'investment_2', 'investment_3','investment_4']] = train[['investment_1', 'investment_2', 'investment_3','investment_4']]
test_init[['investment_1', 'investment_2', 'investment_3','investment_4']] = test[['investment_1', 'investment_2', 'investment_3','investment_4']]

In [21]:
train_init = train_init.fillna(0)
test_init = test_init.fillna(0)

In [22]:
train_init.loc[train_init.gender=='M','gender']=1
train_init.loc[train_init.gender=='F','gender']=0
test_init.loc[test_init.gender=='M','gender']=1
test_init.loc[test_init.gender=='F','gender']=0

In [69]:
test_init.columns

Index(['id', 'age', 'gender', 'cc_cons_apr', 'cc_cons_may', 'cc_cons_jun',
       'card_lim', 'con_avg_all', 'con_avg_jun_may', 'con_avg_apr_may',
       'con_avg_of_avg', 'cc_con_std_all', 'transactions_jun',
       'transactions_may', 'transactions_apr', 'transactions_count_jun',
       'transactions_count_may', 'transactions_count_apr', 'transactions_freq',
       'transactions_debit_freq', 'transactions_credit_freq', 'investment_1',
       'investment_2', 'investment_3', 'investment_4'],
      dtype='object')

In [70]:
train_saved = pd.concat([train.drop(['card_lim'],axis=1),train_init[['card_lim', 'con_avg_all', 'con_avg_jun_may',
                                                                    'con_avg_apr_may', 'con_avg_of_avg', 'cc_con_std_all',
                                                                    'transactions_jun', 'transactions_may', 'transactions_apr',
                                                                    'transactions_count_jun','transactions_count_may', 
                                                                    'transactions_count_apr', 'transactions_freq',
                                                                    'transactions_debit_freq', 'transactions_credit_freq']]],axis=1)
test_saved = pd.concat([test.drop(['card_lim'],axis=1),test_init[['card_lim', 'con_avg_all', 'con_avg_jun_may',
                                                                    'con_avg_apr_may', 'con_avg_of_avg', 'cc_con_std_all',
                                                                    'transactions_jun', 'transactions_may', 'transactions_apr',
                                                                    'transactions_count_jun','transactions_count_may', 
                                                                    'transactions_count_apr', 'transactions_freq',
                                                                    'transactions_debit_freq', 'transactions_credit_freq']]],axis=1)


In [71]:
train_saved.to_csv('train_csv_derived.csv',index=False)
test_saved.to_csv('test_csv_derived.csv',index=False)

In [25]:
def rmsle_cv(model,df_train,df_target):
    kf = KFold(n_splits=5, shuffle=True, random_state=6).get_n_splits(df_train.values)
    rmse= np.sqrt(-cross_val_score(model, df_train.values, df_target.values, scoring="neg_mean_squared_error", cv = kf))
    print(rmse)
    return(rmse)

In [26]:
lgbmr = lgbm.LGBMRegressor(n_estimators=30,learning_rate=0.05)


In [27]:
rmsle_cv(lgbmr,train_init.drop(['id'],axis=1),train['cc_cons'])

[12501.01087253 10679.76974182 11446.0460268  10570.65392703
 10560.5289351 ]


array([12501.01087253, 10679.76974182, 11446.0460268 , 10570.65392703,
       10560.5289351 ])

In [60]:
lr = LinearRegression()

In [61]:
rmsle_cv(lr,train_init.drop(['id'],axis=1),train['cc_cons'])

[12126.07497128 10604.06950058 11254.987249   10379.78116868
 10588.3694023 ]


array([12126.07497128, 10604.06950058, 11254.987249  , 10379.78116868,
       10588.3694023 ])

In [104]:
lr.fit(train_init.drop(['id'],axis=1),train['cc_cons'])

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=True)

In [58]:
rdg = Ridge(normalize=True,alpha=0.005,solver='cholesky',tol=0.0001)

In [59]:
rmsle_cv(rdg,train_init.drop(['id'],axis=1),train['cc_cons'])

[12124.47536744 10603.15023866 11256.54782646 10378.76785428
 10583.865737  ]


array([12124.47536744, 10603.15023866, 11256.54782646, 10378.76785428,
       10583.865737  ])

In [62]:
rdg.fit(train_init.drop(['id'],axis=1),train['cc_cons'])

Ridge(alpha=0.005, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=True, random_state=None, solver='cholesky', tol=0.0001)

In [106]:
submission1 = pd.DataFrame({'id':test.id,'cc_cons':lr.predict(test_init.drop(['id'],axis=1))})

In [63]:
submission2 = pd.DataFrame({'id':test.id,'cc_cons':rdg.predict(test_init.drop(['id'],axis=1))})

In [64]:
test_init[submission2.cc_cons<0]

,id,age,gender,cc_cons_apr,cc_cons_may,cc_cons_jun,card_lim,con_avg_all,con_avg_jun_may,con_avg_apr_may,con_avg_of_avg,cc_con_std_all,transactions_jun,transactions_may,transactions_apr,transactions_count_jun,transactions_count_may,transactions_count_apr,transactions_freq,transactions_debit_freq,transactions_credit_freq,investment_1,investment_2,investment_3,investment_4
10615,5452,33,1,11080.28,2758.0,4390.0,100000.0,6076.093333,3574.0,6919.14,5523.077778,4409.905798,130815.11,147850.94,169273.22,654.0,329.0,488.0,304.51344,343.811556,268.147251,0.0,0.0,0.0,0.0


In [67]:
submission2.loc[submission2.cc_cons<0,'cc_cons']=(2758.0+4390.0)/2

In [68]:
submission2.to_csv('submission_derived_features_rdg.csv',index=False)